In [1]:
import numpy as np
import pandas as pd
import psycopg2 as psy
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sb
import pickle as pic

%matplotlib inline

In [2]:
# Connect to the Lyrics Database
conn = psy.connect("dbname=test")
cur = conn.cursor()

# Put Data in Pandas DataFrame
songs = pd.read_sql_query("SELECT * FROM songs;", conn)
refs = pd.read_sql_query("SELECT * FROM referents;", conn)
anns = pd.read_sql_query("SELECT * FROM annotations;", conn)

In [3]:
# Songs DataFrame Info
print songs.shape
print songs.info()
songs.head()

(9154, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9154 entries, 0 to 9153
Data columns (total 8 columns):
song_id                   9154 non-null int64
hot                       9154 non-null bool
unreviewed_annotations    9154 non-null int64
title                     9154 non-null object
full_title                9154 non-null object
artist                    9154 non-null object
artist_id                 9154 non-null int64
annotation_count          9154 non-null int64
dtypes: bool(1), int64(4), object(3)
memory usage: 509.6+ KB
None


,song_id,hot,unreviewed_annotations,title,full_title,artist,artist_id,annotation_count
0,2890384,False,0,0-100,0-100 by John Nonny,John Nonny,1020738,0
1,156640,False,0,0 to 100 / The Catch Up,0 to 100 / The Catch Up by Drake,Drake,130,41
2,2136824,False,0,100,100 by The Game (Ft. Drake),The Game,42,43
3,2688225,False,0,100it Racks,"100it Racks by DJ Esco (Ft. 2 Chainz, Drake & ...",DJ Esco,49470,50
4,703738,False,2,10 Bands,10 Bands by Drake,Drake,130,30


In [4]:
# Referent DataFrame Info
print refs.shape
print refs.info()
refs.head()

(115978, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115978 entries, 0 to 115977
Data columns (total 6 columns):
id                115978 non-null int64
song_id           115978 non-null int64
classification    115978 non-null object
fragment          115956 non-null object
is_description    115978 non-null bool
annotator_id      115978 non-null int64
dtypes: bool(1), int64(3), object(2)
memory usage: 4.5+ MB
None


,id,song_id,classification,fragment,is_description,annotator_id
0,4961787,156640,accepted,[Part I: 0 to 100],False,605899
1,3274596,156640,accepted,"[Produced by Boi-1da, Frank Dukes, Noah ""40"" S...",False,104344
2,3272685,156640,accepted,"Maybe I'm searchin' for the problems, askin' w...",False,58812
3,3272333,156640,accepted,"The other night, Lavish Lee told me that I'm a...",False,658401
4,3272181,156640,accepted,[Bridge]\n Whole squad on that real shit\n Who...,False,18490


In [5]:
# Annotation DataFrame Info
print anns.shape
print anns.info()
anns.head()

(115898, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115898 entries, 0 to 115897
Data columns (total 12 columns):
id               115898 non-null int64
song_id          115898 non-null int64
ref_id           115898 non-null int64
ann_text         115898 non-null object
verified         115898 non-null bool
cosigned_by      115898 non-null object
has_voters       115898 non-null bool
state            115898 non-null object
community        115898 non-null bool
pinned           115898 non-null bool
comment_count    115898 non-null int64
votes_total      115898 non-null int64
dtypes: bool(4), int64(5), object(3)
memory usage: 7.5+ MB
None


,id,song_id,ref_id,ann_text,verified,cosigned_by,has_voters,state,community,pinned,comment_count,votes_total
0,4961787,156640,4961787,This song was allegedly supposed to be Diddy’s...,False,{},True,accepted,True,False,0,11
1,3274596,156640,3274596,https://twitter.com/Boi1da/status/473262859418...,False,{},True,accepted,True,False,0,56
2,3272685,156640,3272685,"Like he says in “Think Good,” Drake is constan...",False,{},True,accepted,True,False,0,19
3,3272333,156640,3272333,Lavish Lee is the best friend of Melissa Shay ...,False,{},True,accepted,True,False,0,37
4,3272181,156640,3272181,Drake’s only squads are OVO (and TOPSZN lowkey...,False,{},True,accepted,True,False,0,44


In [6]:
# Change UTF-8 to ASCII
# Define Genius Decoder Function
def genius_decoder(x):
    if x:
        return x.decode('utf8').encode('ascii', 'ignore')
    else:
        return x

# Decode Referents and Annotations    
refs['fragment'] = refs['fragment'].apply(genius_decoder)
anns['ann_text'] = anns['ann_text'].apply(genius_decoder)

In [43]:
# Merge Referent and Song Info
ref_song = pd.merge(songs, refs, how='right', left_on='song_id', right_on='song_id')

# Drop Unnecessary Columns
ref_song.drop(['hot', 'unreviewed_annotations', 'full_title', 'annotation_count'], axis=1, inplace=True)

# Merge in Annotations to Dataset
all_data = pd.merge(ref_song, anns, how='left', left_on='id', right_on='ref_id')
print all_data.columns

# Drop Unnecessary Columns
all_data.drop(['verified', 'cosigned_by', 'state', 'community', 'pinned', 'song_id_y', 'ref_id'], axis=1, inplace=True)
all_data.rename(columns={"song_id_x": "song_id", "id_x": "ref_id", "id_y": "ann_id"}, inplace=True)

# Drop Row with Missing Data ~80
all_data.dropna(inplace=True)
print all_data.info()

# Drop Rows where fragment length = 0
all_data = all_data[all_data['fragment'].apply(lambda x: len(x) > 0)]

# Drop Rows where annotation length = 0
all_data = all_data[all_data['ann_text'].apply(lambda x: len(x) > 0)]
print all_data.info()

Index([u'song_id_x', u'title', u'artist', u'artist_id', u'id_x',
       u'classification', u'fragment', u'is_description', u'annotator_id',
       u'id_y', u'song_id_y', u'ref_id', u'ann_text', u'verified',
       u'cosigned_by', u'has_voters', u'state', u'community', u'pinned',
       u'comment_count', u'votes_total'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 115876 entries, 0 to 115979
Data columns (total 14 columns):
song_id           115876 non-null int64
title             115876 non-null object
artist            115876 non-null object
artist_id         115876 non-null int64
ref_id            115876 non-null int64
classification    115876 non-null object
fragment          115876 non-null object
is_description    115876 non-null bool
annotator_id      115876 non-null int64
ann_id            115876 non-null float64
ann_text          115876 non-null object
has_voters        115876 non-null object
comment_count     115876 non-null float64
votes_total     

In [44]:
# Drop Rows where Fragment are titles, headers, or notes instead of actual lyrics
def is_lyrics(frag):
    if frag[0] == "[" and frag[-1] == "]":
        return False
    else:
        return True

all_data = all_data[all_data['fragment'].apply(is_lyrics)]
print all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111008 entries, 2 to 115979
Data columns (total 14 columns):
song_id           111008 non-null int64
title             111008 non-null object
artist            111008 non-null object
artist_id         111008 non-null int64
ref_id            111008 non-null int64
classification    111008 non-null object
fragment          111008 non-null object
is_description    111008 non-null bool
annotator_id      111008 non-null int64
ann_id            111008 non-null float64
ann_text          111008 non-null object
has_voters        111008 non-null object
comment_count     111008 non-null float64
votes_total       111008 non-null float64
dtypes: bool(1), float64(3), int64(4), object(6)
memory usage: 12.0+ MB
None


In [61]:
# Drop Data from Artists below 1st quartile in # of references
refs_by_artist = all_data.groupby('artist_id').size()
drop_artist_ids = list(refs_by_artist[songs_by_artist < 39].index)
drop_artist_mask = all_data['artist_id'].apply(lambda x: x not in drop_artist_ids)
all_data = all_data[drop_artist_mask]
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103882 entries, 2 to 115979
Data columns (total 14 columns):
song_id           103882 non-null int64
title             103882 non-null object
artist            103882 non-null object
artist_id         103882 non-null int64
ref_id            103882 non-null int64
classification    103882 non-null object
fragment          103882 non-null object
is_description    103882 non-null bool
annotator_id      103882 non-null int64
ann_id            103882 non-null float64
ann_text          103882 non-null object
has_voters        103882 non-null object
comment_count     103882 non-null float64
votes_total       103882 non-null float64
dtypes: bool(1), float64(3), int64(4), object(6)
memory usage: 11.2+ MB


In [64]:
# Pickle our data
pic.dump(all_data, open("../Data/all_data.p", "wb"))